# Imports

In [ ]:
import utils.data_prc as dp 
import utils.build_network as bn
import utils.compilation_opt as cpo
%pylab inline

In [ ]:
dp.dataset_pickle_emotions("fer2013.csv", False)
dp.dataset_pickle_pain("pain_ds")

In [ ]:
X_train_E, y_train_E, X_test_E, y_test_E = dp.dataset_loading_emotion("fer2013")
X_train, y_train, X_test, y_test, X_val, y_val = dp.dataset_loading_pain("pain_ds")

In [ ]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape, X_val.shape, y_val.shape)
print(X_train_E.shape, y_train_E.shape, X_test_E.shape, y_test_E.shape)

In [ ]:
emotion = { 0:'Angry', 1:'Disgust', 2: 'Fear', 3: 'Happy',
           4: 'Sad', 5: 'Surprise', 6: 'Neutral'}

In [ ]:
num_classes = len(emotion)
samples_per_class = 7

for y, cls in emotion.items():
    idxs = np.flatnonzero(y_train_E == y)
    idxs = np.random.choice(idxs, samples_per_class, replace=True)
    for i, idx in enumerate(idxs):
        plt_idx = i * num_classes + y + 1
        plt.subplot(samples_per_class, num_classes, plt_idx)
        plt.imshow(reshape(X_train_E[idx], (48, 48)).astype('uint8'))
        plt.axis('off')
        if i == 0:
            plt.title(cls)

plt.show()


In [ ]:
NUM_CLASSES_EMOTION = 7
NUM_CLASSES_PAIN = 16

#bn.print_metadata(X_train, X_test)
y_train_E, y_test_E = dp.y_to_categorical(y_train_E, y_test_E, NUM_CLASSES_EMOTION)
y_train, y_test, y_val = dp.y_to_categorical(y_train, y_test, NUM_CLASSES_PAIN, y_val)

In [ ]:
#opt = cpo.sgdOpt()
#opt = cpo.rmsPropOpt()
#opt = cpo.adagradOpt() 
opt = cpo.adamOpt() 

#X_train_E, X_test_E = dp.prepare_emotions_examples(X_train_E, X_test_E)

model = bn.build_model(X_train)

model = cpo.compiling(model, opt)
model.summary()

In [ ]:
epochs = 5
batch_size = 128

#print (X_train.shape, y_train.shape)
#model, history = cpo.training(model, batch_size, epochs, X_train_E, y_train_E)
model, history = cpo.training_painds(model, batch_size, epochs, X_train, y_train, X_val, y_val)

In [ ]:
cpo.eval_plot(model, X_train, y_train, history, epochs)

In [ ]:
#cpo.eval_plot(model, X_test, y_test, history, epochs)

In [ ]:
#print(model.predict_classes(X_train, batch_size=128))

In [ ]:
#print(model.predict_classes(X_test, batch_size=128))

In [ ]:
from keras.models import load_model

model.save('my_model_no_reg_pain.h5')  # creates a HDF5 file '.h5'
#del model  # deletes the existing model

# returns a compiled model
# identical to the previous one
#model = load_model('my_model.h5')